In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [71]:
from query.query_functions2 import get_result_from_triples
from copy import copy, deepcopy

In [64]:
from loaders.loaders import *
lc_quad_train = load_on_path(ds_lc_quad_train_cleaned)

In [80]:
def is_ontology_property_replacable(record):
    original_results = get_result_from_triples(record["graph"])

    ontology_only_query = []
    for i, triple in enumerate(record["graph"]):
        triple_copy = deepcopy(triple)
        if "property" in triple_copy[1]:
            triple_copy[1] = triple_copy[1].replace("property", "ontology")
        ontology_only_query.append(triple_copy)

    ontology_only_results = get_result_from_triples(ontology_only_query)

    property_only_query = []
    for i, triple in enumerate(record["graph"]):
        triple_copy = deepcopy(triple)
        if "ontology" in triple_copy[1]:
            triple_copy[1] = triple_copy[1].replace("ontology", "property")
        property_only_query.append(triple_copy)

    property_only_results = get_result_from_triples(property_only_query)

    if ontology_only_results is None:
        vs_ontology_flag = False
    else:
        vs_ontology_only_difference = set(original_results).intersection(set(ontology_only_results))
        vs_ontology_flag = (len(vs_ontology_only_difference) == len(set(original_results))) and (len(original_results) == len(ontology_only_results))

    if property_only_results is None:
        vs_property_flag = False
    else:
        vs_property_only_difference = set(original_results).intersection(set(property_only_results))
        vs_property_flag = (len(vs_property_only_difference) == len(set(original_results))) and (len(original_results) == len(property_only_results))

    return [vs_property_flag, vs_ontology_flag]

In [81]:
prop_ont_replacable_flags = list()
for i, record in enumerate(lc_quad_train):
    print(i)
    prop_ont_replacable_flags.append(is_ontology_property_replacable(record))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [83]:
from collections import Counter
cnt_prop = Counter(list(map(lambda x: x[0], prop_ont_replacable_flags)))
cnt_ont = Counter(list(map(lambda x: x[1], prop_ont_replacable_flags)))

In [84]:
cnt_prop

Counter({False: 171, True: 79})

In [85]:
cnt_ont

Counter({False: 137, True: 113})

# Stage

In [47]:
# Single
record = lc_quad_train[6]
original_results = get_result_from_triples(record["graph"])

ontology_only_query = []
for i, triple in enumerate(record["graph"]):
    triple_copy = copy(triple)
    if "property" in triple_copy[1]:
        triple_copy[1] = triple_copy[1].replace("property", "ontology")
    ontology_only_query.append(triple_copy)

ontology_only_results = get_result_from_triples(ontology_only_query)
print("AYE:", ontology_only_query)
print("AYE:", ontology_only_results)

property_only_query = []
for i, triple in enumerate(record["graph"]):
    triple_copy = copy(triple)
    if "ontology" in triple_copy[1]:
        triple_copy[1] = triple_copy[1].replace("ontology", "property")
    property_only_query.append(triple_copy)

property_only_results = get_result_from_triples(property_only_query)

if ontology_only_results is None:
    vs_ontology_flag = False
else:
    vs_ontology_only_difference = set(original_results).intersection(set(ontology_only_results))
    vs_ontology_flag = (len(vs_ontology_only_difference) == len(set(original_results))) and (len(original_results) == len(ontology_only_results))

if property_only_results is None:
    vs_property_flag = False
else:
    vs_property_only_difference = set(original_results).intersection(set(property_only_results))
    vs_property_flag = (len(vs_property_only_difference) == len(set(original_results))) and (len(original_results) == len(property_only_results))

print([vs_property_flag, vs_ontology_flag])

AYE: [['?uri', '<http://dbpedia.org/ontology/formerTeam>', '<http://dbpedia.org/resource/Indianapolis_Colts>'], ['?uri', '<http://dbpedia.org/ontology/formerTeam>', '<http://dbpedia.org/resource/Carolina_Panthers>']]
AYE: ['http://dbpedia.org/resource/Jim_Harbaugh', 'http://dbpedia.org/resource/Kerry_Collins', 'http://dbpedia.org/resource/Josh_Vaughan', 'http://dbpedia.org/resource/Mike_Pollak', 'http://dbpedia.org/resource/Bryan_Jurewicz', 'http://dbpedia.org/resource/Matt_Blanchard', 'http://dbpedia.org/resource/Drake_Nevis', 'http://dbpedia.org/resource/Kevin_McMahan', 'http://dbpedia.org/resource/Mike_Seidman', 'http://dbpedia.org/resource/Jim_Kubiak', 'http://dbpedia.org/resource/Jerome_Felton', 'http://dbpedia.org/resource/Tupe_Peko', 'http://dbpedia.org/resource/Pat_Ross_(American_football)', 'http://dbpedia.org/resource/Thomas_Austin_(American_football)', 'http://dbpedia.org/resource/Tanard_Davis', 'http://dbpedia.org/resource/Jason_Shirley', 'http://dbpedia.org/resource/Tauren

TypeError: 'NoneType' object is not iterable